In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import surprise as sp
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV, RandomizedSearchCV
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, SVD, NMF
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tqdm import tqdm, tqdm_notebook
import time
import os

## Comparativa Metodos KNN y Busqueda de hiperParametros

In [17]:
df_train = pd.read_csv('./data/ratings_train.csv')
df_test = pd.read_csv('./data/ratings_test.csv')

In [18]:
#trainset, testset = train_test_split(df_train, test_size=0.30)
trainset = df_train
testset = df_test
print(trainset.shape, testset.shape)

(770089, 10) (85509, 10)


In [4]:
#trainset.isnull().sum()

## HiperParametros

In [19]:
reader = sp.reader.Reader(rating_scale=(0.5, 5))
data_train= sp.dataset.Dataset.load_from_df(df_train[['userID', 'movieID', 'rating']], reader)


# opciones del algoritmo
param_grid_svd = {'n_factors': [80, 90, 100, 110, 120], 'n_epochs': [10, 15, 20, 25, 30], 'lr_all': [0.001, 0.003, 0.005, 0.008],
              'reg_all': [0.01, 0.02, 0.03]}

param_grid_nmf = {'n_factors': [5, 10, 15, 20, 25], 'n_epochs': [40, 50, 60]}

SVD_algo = SVD()
NMF_algo = NMF()
# Run 5-fold cross-validation and print results
#cross_validate(SVD_algo, data_train, measures=['RMSE', 'MAE'], cv=5, verbose=True)
#print("======================================")
#cross_validate(NMF_algo, data_train, measures=['RMSE', 'MAE'], cv=5, verbose=True)


In [5]:
#gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
#gs.fit(data)
#print("SVD")
#print('RMSE =', gs.best_params['rmse']) #, 'MAE =', knnbasic_gs.best_params['mae'])

In [6]:
t_0 = time.time()
svd_rs = RandomizedSearchCV(SVD, param_grid_svd, measures=['rmse', 'mae'],n_iter=10, n_jobs=-1)
svd_rs.fit(data_train)
print("SVD")
print('RMSE =', svd_rs.best_params['rmse']) 
print ("tiempo: ", time.time() - t_0)

SVD
RMSE = {'n_factors': 110, 'n_epochs': 10, 'lr_all': 0.008, 'reg_all': 0.03}
tiempo:  1035.5437660217285


In [7]:
t_0 = time.time()
nmf_rs = RandomizedSearchCV(NMF, param_grid_nmf, measures=['rmse', 'mae'],n_iter=10, n_jobs=-1)
nmf_rs.fit(data_train)
print("NMF")
print('RMSE =', nmf_rs.best_params['rmse']) 
print ("tiempo: ", time.time() - t_0)

NMF
RMSE = {'n_factors': 15, 'n_epochs': 60}
tiempo:  1071.4651556015015


In [20]:
#bsl_svd_options = {'n_factors': 90, 'n_epochs': 15, 'lr_all': 0.005, 'reg_all': 0.03 }
#bsl_nmf_options = {'n_factors': 20, 'n_epochs': 40 }

SVD_algo = SVD(n_factors= 110, n_epochs= 10, lr_all= 0.008, reg_all= 0.03)
NMF_algo = NMF(n_factors= 15, n_epochs= 60)

In [21]:
trainset_model = data_train.build_full_trainset()

In [22]:
t_0 = time.time()
pred_SVD = SVD_algo.fit(trainset_model)
print ("tiempo: ", time.time() - t_0)

tiempo:  27.544204235076904


In [23]:
t_0 = time.time()
pred_NMF = NMF_algo.fit(trainset_model)
print ("tiempo: ", time.time() - t_0)

tiempo:  69.26765632629395


In [24]:
print(len(testset))

85509


In [25]:
def full_prediction(algo_name,algoritmo,test):
    list_pred = []
    for index, row in tqdm_notebook(test.iterrows()):
        row_pred = dict()
        uid = row.userID
        iid = row.movieID
        pred = algoritmo.predict(uid, iid, verbose=False)
        row_pred["ID"] = row.ID.astype(int)
        row_pred["rating"] = np.around(pred.est, 2)
        list_pred.append(row_pred)
    print(row.ID.astype(int))
    df = pd.DataFrame(list_pred)    
    return list_pred
    #print(algo_name + " MSE:      %.4f" % mean_squared_error(test.rating, df.rating))

In [28]:
df_list_pred_SVD = pd.DataFrame(full_prediction("SVD",SVD_algo,testset))
df_list_pred_NMF = pd.DataFrame(full_prediction("NMF",NMF_algo,testset))

#full_prediction("NMF",NMF_algo,testset)


855571



855571


In [29]:
df_list_pred_SVD.to_csv("./results/ignacio_submit_SVD.csv", index=False)
df_list_pred_NMF.to_csv("./results/ignacio_submit_MSE.csv", index=False)